<a href="https://colab.research.google.com/github/2SUBDA/Breakouts/blob/Week5/SOTUS2HardTryNewsGroups.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sample NLP Code for IST 718



In [0]:
# A quick sample for using Universal Language Model Fine-Tuning for Text Classification
# aka ULMFiT
# Check out nlp.fast.ai for more

# import libraries
import fastai
from fastai import *
from fastai.text import * 
import pandas as pd
import numpy as np
from functools import partial
import io
import os

In [0]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu92/torch_nightly.html
!pip install fastai

In [3]:
# OBTAIN

from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

In [0]:
# OBTAIN

df = pd.DataFrame({'label':dataset.target, 'text':dataset.data})

In [5]:
df.shape

(11314, 2)

Let's try looking at just Politics and Windows articles - source?  https://kdd.ics.uci.edu/databases/20newsgroups/20newsgroups.data.html


In [0]:
# SCRUB

df = df[df['label'].isin([5,18])]
df = df.reset_index(drop = True)

In [7]:
df['label'].value_counts()


5     593
18    465
Name: label, dtype: int64

In [0]:
# SCRUB - TEXT

df['text'] = df['text'].str.replace("[^a-zA-Z]", " ")


In [9]:
# SCRUB - STOPWORDS
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords 
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
# SCRUB - TOKEN

# tokenization 
tokenized_doc = df['text'].apply(lambda x: x.split())

# remove stop-words 
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])

# de-tokenization 
detokenized_doc = [] 
for i in range(len(df)): 
    t = ' '.join(tokenized_doc[i]) 
    detokenized_doc.append(t) 

df['text'] = detokenized_doc

In [0]:
# SCRUB - TEST & TRAIN

from sklearn.model_selection import train_test_split

# split data into training and validation set
df_trn, df_val = train_test_split(df, stratify = df['label'], test_size = 0.4, random_state = 12)

In [12]:
# SCRUB - CHECK SPLIT

df_trn.shape, df_val.shape

((634, 2), (424, 2))

In [0]:
# MODEL

# Language model data
data_lm = TextLMDataBunch.from_df(train_df = df_trn, valid_df = df_val, path = "")

# Classifier model data
data_clas = TextClasDataBunch.from_df(path = "", train_df = df_trn, valid_df = df_val, vocab=data_lm.train_ds.vocab, bs=32)

In [0]:
# MODEL - TUNE PRE-TRAINED MODEL

learn = language_model_learner(data_lm, pretrained_model=URLs.WT103, drop_mult=0.7)

In [15]:
# MODEL - LEARNING RATE

# train the learner object with learning rate = 1e-2
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,7.862900,6.350390,0.144460


In [0]:
# MODEL - SAVE

learn.save_encoder('ft_enc')

In [0]:
# MODEL - BUILD CLASSIFIER

learn = text_classifier_learner(data_clas, drop_mult=0.7)
learn.load_encoder('ft_enc')

In [18]:
# MODEL - FIT

learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy
1,0.536539,0.484171,0.797170


In [19]:
# MODEL - EVALUATE

preds, targets = learn.get_preds()

predictions = np.argmax(preds, axis = 1)
pd.crosstab(predictions, targets)

col_0,0,1
row_0,,
0,169,17
1,69,169
